Import libraries


In [49]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import SVC 

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR, NuSVR, SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr

import scipy
import random
import csv
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable

Import Dataset (from organiser's github page  https://github.com/MMU-TDMLab/CompLex)

In [50]:
%%bash
rm -r sample_data

mkdir train
cd train
wget -q 'https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/train/lcp_single_train.tsv?dl=1' -O 'lcp_single_train.tsv'
wget -q 'https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/train/lcp_multi_train.tsv?dl=1' -O 'lcp_multi_train.tsv'
cd ..

mkdir trial
cd trial
wget -q 'https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/trial/lcp_single_trial.tsv' -O 'lcp_single_trial.tsv'
wget -q 'https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/trial/lcp_multi_trial.tsv' -O 'lcp_multi_trial.tsv'
cd ..

mkdir test
cd test
wget -q 'https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/test/lcp_single_test.tsv?dl=1' -O 'lcp_single_test.tsv'
wget -q 'https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/test/lcp_multi_test.tsv?dl=1' -O 'lcp_multi_test.tsv'
cd ..

mkdir test-labels
cd test-labels
wget -q 'https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/test-labels/lcp_single_test.tsv?dl=1' -O 'lcp_single_test.tsv'
wget -q 'https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/test-labels/lcp_multi_test.tsv?dl=1' -O 'lcp_multi_test.tsv'
cd ..

mkdir score
mkdir Embeddings
# cd Embeddings
# wget -q 'https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/test-labels/lcp_single_test.tsv?dl=1' -O 'lcp_single_test.tsv'
# cd ..

mkdir saves

rm: cannot remove 'sample_data': No such file or directory
mkdir: cannot create directory ‘train’: File exists
mkdir: cannot create directory ‘trial’: File exists
mkdir: cannot create directory ‘test’: File exists
mkdir: cannot create directory ‘test-labels’: File exists
mkdir: cannot create directory ‘score’: File exists
mkdir: cannot create directory ‘Embeddings’: File exists
mkdir: cannot create directory ‘saves’: File exists


Initialise torch for deep nn

In [51]:
torch.backends.cudnn.deterministic = True
torch.manual_seed(123)
torch.cuda.manual_seed_all(123)
np.random.seed(123)
torch.cuda.manual_seed_all(123)

In [52]:
original_to_standard = {}
original_to_standard[0]=0
original_to_standard[0.25]=1
original_to_standard[0.5]=2
original_to_standard[0.75]=3
original_to_standard[1]=4
standard_to_original = {}
standard_to_original[0]=0
standard_to_original[1]=0.25
standard_to_original[2]=0.5
standard_to_original[3]=0.75
standard_to_original[4]=1

Input electra embedding

In [53]:
f = open('Embeddings/Electra_embeddings.json') 

electra_embeddings = json.load(f) 

Read train and test data for single words

In [54]:
df_train = pd.read_csv("train/lcp_single_train.tsv",
                              delimiter="\t", quoting=csv.QUOTE_NONE, encoding='utf-8')

df_test = pd.read_csv("test/lcp_single_test.tsv",
                              delimiter="\t", quoting=csv.QUOTE_NONE, encoding='utf-8')

In [ ]:
df_test.head()

Preprocess data; get embeddings for words
ELECTRA Embeddings

In [56]:
token = df_train['id'].values
complexity = df_train['complexity'].values

In [57]:
def append_electra(tokens, embeddings, complexity):
    x_train = []
    for i in range(len(tokens)):
        temp = np.array(embeddings[tokens[i]])
        x_train.append(temp)
    # print(f'from get_train_electra output train data {x_train[20]}')
    x_train = np.array(x_train, dtype=np.float32)
    y_train = complexity
    
    return x_train, y_train

In [58]:
def mae(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs(actual - pred))

Classification task, get annoations, preporcess data

In [59]:
def generate_annotations(df,n=20):   
    scores = df['complexity']
    annotations = []
    for i in range(len(scores)):
        if scores[i]>=0 and scores[i]<0.25:
            low = 0
            high = 0.25
        elif scores[i]>=0.25 and scores[i]<0.5:
            low = 0.25
            high = 0.5
        elif scores[i]>=0.5 and scores[i]<0.75:
            low = 0.5
            high = 0.75
        elif scores[i]>=0.75 and scores[i]<1:
            low = 0.75
            high = 1
        alpha = (high - scores[i])/(high-low)
        num_low = int(np.floor(alpha*n))
        num_high = n - num_low

        temp = int(original_to_standard[high])*np.ones(n,dtype=int)
        # for i in range(num_low):
        #     temp[i]=int(original_to_standard[low])
        annotations.append(temp.tolist())
        print(num_low, num_high)
    annotations = np.array(annotations)
    
    return annotations

def zscore(x, m, s):
    for i in range(len(x[0,:])):
        x[:,i] = (x[:,i] - m[i])/s[i]
    return x

def getx_train(tokens, embeddings_dict):
    x_train = []
    for i in range(len(tokens)):
        temp = np.array(embeddings_dict[tokens[i]])
        x_train.append(temp)

    x_train = np.array(x_train, dtype = np.float32)
    return x_train

def classification(x, y):   
    # clf = SVC(kernel = 'linear', verbose=1, C = 1).fit(x, y) 
    # clf = SVC(kernel = 'rbf', verbose=1,C = 1).fit(x, y)

    # if model == 'svc':
    clf = SVC(kernel = 'sigmoid', verbose=1, C = 1).fit(x, y)
      # return clf
    # if model == 'rfc':
    clf=RandomForestClassifier(max_depth = 18, verbose=1).fit(x, y)
      # return clf
    # clf = KNeighborsClassifier(n_neighbors = 5, verbose=1).fit(x, y) 
    # if model == 'lr':
    clf=LogisticRegression(random_state=0, verbose=1).fit(x, y)
      # return clf
    # if model == 'gnb':
    # gnb = GaussianNB(verbose=1).fit(x, y) 
    print(f'from classification function {clf}')
    
    return clf

In [60]:
def append_annotaions(df_train,df_test, embeddings_dict1, n=50):
    annotations = generate_annotations(df_train)
    print(annotations)
    x_train = getx_train(df_train['id'], embeddings_dict1)
    x_test = getx_train(df_test['id'], embeddings_dict1)
    for i in range(n):
        y_train = annotations[:,i]

    return x_train,x_test,y_train

In [61]:
x_train_r, y_train_r = append_electra(token, electra_embeddings, complexity)

In [ ]:
x_train_c, x_test_c, y_train_c = append_annotaions(df_train, df_test, electra_embeddings, 50)

In [62]:
x_train_c, x_test_c, y_train_c

(array([[ 0.06197025,  0.861919  , -0.6912668 , ..., -0.20722966,
         -0.02224979,  0.3467312 ],
        [-0.05221076,  0.49938086,  0.12782227, ...,  0.02429406,
          0.41544113,  0.32608533],
        [ 0.28723556,  0.8429072 ,  0.20104708, ...,  0.33296737,
          0.4963683 ,  0.42973074],
        ...,
        [-0.41140014,  0.10154386,  0.22980098, ...,  0.5838036 ,
          0.22408064,  0.3742459 ],
        [ 0.8498829 ,  0.40331137, -0.15938659, ...,  0.8757806 ,
         -0.05694856, -0.06274758],
        [-0.42602527, -0.11619602,  0.0687228 , ...,  0.3005255 ,
         -0.6548657 , -0.23669678]], dtype=float32),
 array([[ 0.02976113,  0.06232207, -0.49189216, ...,  0.9299774 ,
         -0.44129634,  0.38640308],
        [-0.38132754,  0.21804713,  0.17884642, ...,  0.15081623,
          0.9958955 ,  0.17070249],
        [ 0.4999454 ,  0.32943156,  0.03449274, ...,  0.07638318,
         -0.42761996,  0.23136139],
        ...,
        [-0.39636996,  0.22865525,  0.6

In [ ]:
mae(y_test, y_pred)

Train Model

In [ ]:
# def train_regression(x_train, y_train):
#     model = LinearRegression().fit(x_train, y_train) 
#     # model = LogisticRegression(random_state=0, verbose=1).fit(x_train, y_train)       
#     return model

In [ ]:
# model = train_regression(x_train_r, y_train_r)

Embed test set for regression prediction


In [ ]:
# x_test, y_test = append_electra(token, electra_embeddings, complexity)

In [ ]:
def predict(model, x):
    predicted = model.predict(x)
    # print(predicted)
    predictions = []

    # pred = np.array(predictions, dtype = float)
    # for i in range(len(pred)):
    #     for j in range(len(pred[i])):
    #         pred[i,j]=standard_to_original[int(pred[i,j])]
            
    # y_pred = np.zeros(y_test.shape,dtype=float)
    # for j in range(len(y_test)):
    #     y_pred[j] = pred[:,j].mean()
    
    return predicted

In [ ]:
# predictions = predict(model, x_test)

In [47]:
def generate_scores(split, filename):
    submission_dict = {}
    with open(filename, 'r') as sf:
        csv_reader = csv.reader(sf)
        for item in csv_reader:
            try:
                submission_dict[item[0]] = float(item[1])
            except:
                # print(item)
                pass

    #Read reference:
    gold_dict = {}
    if split=='single test':
        path='test-labels/lcp_single_test.tsv'
    else:
        path='test-labels/lcp_multi_test.tsv'

    reference_file = pd.read_csv(path,  delimiter="\t", quoting=csv.QUOTE_NONE, encoding='utf-8')
    for id, complexity in zip(list(reference_file.id), list(reference_file.complexity)):
        gold_dict[id] = float(complexity)

    #Produce vectors with reference labels and predictions:
    gold = []
    predicted = []
    
    for key, label in gold_dict.items():
        gold.append(label)
        predicted.append(submission_dict[key])
    
    #Calculate scores:
    pearson_score = pearsonr(gold, predicted)[0]
    # spearman_score = spearmanr(gold, predicted)[0]
    # mae_score = mean_absolute_error(gold, predicted)
    # mse_score = mean_squared_error(gold, predicted)
    # rsq_score = r2_score(gold, predicted)

    # result = ''
    # result += "pearson:{0}\n".format(pearson_score)
    # result += "spearman:{0}\n".format(spearman_score)
    # result += "mae:{0}\n".format(mae_score)
    # result += "mse:{0}\n".format(mse_score)
    # result += "r2:{0}".format(rsq_score)

    # # print(result)
    # with open('score/scores_'+split+'.txt', 'w') as output_file:
    #     output_file.write(result)

    return pearson_score

In [ ]:
df_test['complexity'] = 0.00000
token = df_test['id'].values
complexity = df_test['complexity'].values

In [ ]:
# model_results = {}

# # log_reg = LinearRegression(n_jobs=-1, verbose=1, random_state=42)
# model = LinearRegression().fit(x_train_r, y_train_r)

# x_test, y_test = append_electra(token, electra_embeddings, complexity)

# predictions, y_test = predict(model, x_test)
# # print(predictions)

# y_pred = 0.5*predictions[1]
# y_pred = list(predictions)

# df_test['Predicted Complexity'] = 0.00000
# for i in range(len(df_test)):
#     df_test['Predicted Complexity'][i] = y_pred[i]

# df_final = df_test.drop(columns=['complexity','corpus','sentence','token'])
# df_final.to_csv('single_test_predictions_LR.csv',index=False,header=None)
# df_final = df_test.drop(columns=['complexity'])
# df_final.to_csv('single_test_predictions_temp_LR.csv',index=False,header=None)

# # evaluate model, return pearson score
# model_result = generate_scores('single test', 'single_test_predictions_LR.csv')
# print(model_result)

Logistics regression

In [ ]:
logistics_reg = LogisticRegression(n_jobs=-1, verbose=1, random_state=0)
model = logistics_reg.fit(x_train_c, y_train_c)

predictions = predict(model, x_test_c)

for i in predictions:
  y_pred = 0.5*predictions[i]
  # print(y_pred)
  y_pred = list(predictions)

df_test['Predicted Complexity'] = 0.00000
for i in range(len(df_test)):
    df_test['Predicted Complexity'][i] = y_pred[i]

df_final = df_test.drop(columns=['complexity','corpus','sentence','token'])
df_final.to_csv('single_test_predictions_LOGR.csv',index=False,header=None)
df_final = df_test.drop(columns=['complexity'])
df_final.to_csv('single_test_predictions_temp_LOGR.csv',index=False,header=None)

# evaluate model, return pearson score
model_result = generate_scores('single test', 'single_test_predictions_LOGR.csv')
print(model_result)

In [ ]:

logistics_reg = LogisticRegression(n_jobs=-1, verbose=1, random_state=42)
# rand_class = SVC(kernel = 'sigmoid', verbose=1, C = 1)
model = logistics_reg.fit(x_train_c, y_train_c)

predictions = predict(model, x_test_c)

y_pred = 0.5*predictions[1]
y_pred = list(predictions)

df_test['Predicted Complexity'] = 0.00000
for i in range(len(df_test)):
    df_test['Predicted Complexity'][i] = y_pred[i]

df_final = df_test.drop(columns=['complexity','corpus','sentence','token'])
df_final.to_csv('single_test_predictions_RAND.csv',index=False,header=None)
df_final = df_test.drop(columns=['complexity'])
df_final.to_csv('single_test_predictions_temp_RAND.csv',index=False,header=None)

# evaluate model, return pearson score
model_result = generate_scores('single test', 'single_test_predictions_RAND.csv')
print(model_result)

[LibSVM]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


0.4936125441491376


In [ ]:
# logistics_reg = LogisticRegression(n_jobs=-1, verbose=1, random_state=42)
rand_class = SVC(kernel = 'sigmoid', verbose=1, C = 1)

model = rand_class.fit(x_train_c, y_train_c)

predictions = predict(model, x_test_c)

y_pred = 0.5*predictions[1]
y_pred = list(predictions)

df_test['Predicted Complexity'] = 0.00000
for i in range(len(df_test)):
    df_test['Predicted Complexity'][i] = y_pred[i]

df_final = df_test.drop(columns=['complexity','corpus','sentence','token'])
df_final.to_csv('single_test_predictions_temp_GNB.csv',index=False,header=None)
df_final = df_test.drop(columns=['complexity'])
df_final.to_csv('single_test_predictions_temp_GNB.csv',index=False,header=None)

# evaluate model, return pearson score
model_result = generate_scores('single test', 'single_test_predictions_temp_GNB.csv')
print(model_result)

In [ ]:
y_pred = 0.5*predictions[1]
y_pred = list(predictions)

df_test['Predicted Complexity'] = 0.00000
for i in range(len(df_test)):
    df_test['Predicted Complexity'][i] = y_pred[i]

df_final = df_test.drop(columns=['complexity','corpus','sentence','token'])
df_final.to_csv('single_test_predictions_3.csv',index=False,header=None)
df_final = df_test.drop(columns=['complexity'])
df_final.to_csv('single_test_predictions_temp_3.csv',index=False,header=None)

Evaluate predictions

In [ ]:
model_result = generate_scores('single test', 'single_test_predictions_3.csv')
print(model_result)

pearson:0.6936367996687073
spearman:0.6595915282977186
mae:0.07127791333886457
mse:0.008491800109336827
r2:0.4753314467498252


In [ ]:

model_results_df = pd.DataFrame(model_result.items(), columns=["Model","Pearson"]).sort_values("Pearson")

ax = sns.barplot(x="Model", y="Pearson", data=model_results_df)
plt.setp(ax.get_xticklabels(), rotation=90);
plt.ylim(.2,.66); 
ax.xaxis.set_tick_params(labelsize=14)
ax.yaxis.set_tick_params(labelsize=14)
ax.set_xlabel('');
ax.set_ylabel('Pearson', fontsize=15);
ax.set_title("Pearson Score for Different Models", fontsize=16);